# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:

# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [8]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,274.70,66,100,18.07,TF,1716997049
1,1,puerto natales,-51.7236,-72.4875,276.40,75,20,8.75,CL,1716997049
2,2,lazaro cardenas,17.9583,-102.2000,304.87,48,4,1.48,MX,1716997049
3,3,grytviken,-54.2811,-36.5092,274.79,91,100,6.02,GS,1716997050
4,4,paracin,43.8608,21.4078,292.75,81,93,3.08,RS,1716997050


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    alpha=.5,
    size = 'Humidity',
    color = 'City'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values
best_humi_max = 60
best_cloud_max = 50

best_cities_data_df = city_data_df.loc[(city_data_df['Cloudiness']<=best_cloud_max)&
                                      (city_data_df['Humidity']<=best_humi_max),:]
# Drop any rows with null values
best_cities_data_df.dropna()
# Display sample data
best_cities_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,lazaro cardenas,17.9583,-102.2000,304.87,48,4,1.48,MX,1716997049
8,8,port alfred,-33.5906,26.8910,294.92,58,6,0.92,ZA,1716997051
9,9,port elizabeth,-33.9180,25.5701,298.31,38,0,0.00,ZA,1716997051
11,11,solleftea,63.1667,17.2667,295.07,33,16,3.89,SE,1716997051
17,17,greeley,40.4233,-104.7091,295.82,59,17,0.89,US,1716996762
...,...,...,...,...,...,...,...,...,...,...
567,567,ciudad lazaro cardenas,17.9583,-102.2000,304.87,48,4,1.48,MX,1716997049
572,572,allariz,42.1904,-7.8018,301.39,53,44,3.38,ES,1716997165
576,576,gao,16.6362,1.6370,317.89,6,3,3.86,ML,1716997165
577,577,bolondron,22.7633,-81.4469,306.26,32,0,2.75,CU,1716997165


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = best_cities_data_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] =''

# Display sample data
hotel_df.head()

/var/folders/dp/ntyts0j948j70jcb6_2mhc040000gn/T/ipykernel_86682/4155195325.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] =''


,City,Country,Lat,Lng,Humidity,Hotel Name
2,lazaro cardenas,MX,17.9583,-102.2000,48,
8,port alfred,ZA,-33.5906,26.8910,58,
9,port elizabeth,ZA,-33.9180,25.5701,38,
11,solleftea,SE,63.1667,17.2667,33,
17,greeley,US,40.4233,-104.7091,59,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
categories = 'accommodation.hotel'
limit = 1

radius = 10000
params = {
    'categories':categories,
    'limit':limit,
    'apiKey':geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index,'Lng']
    latitude = hotel_df.loc[index,'Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
solleftea - nearest hotel: No hotel found
greeley - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
opuwo - nearest hotel: No hotel found
peschiera del garda - nearest hotel: No hotel found
thompson - nearest hotel: No hotel found
yuci - nearest hotel: No hotel found
kruisfontein - nearest hotel: No hotel found
abadeh - nearest hotel: No hotel found
la sarre - nearest hotel: No hotel found
udachny - nearest hotel: No hotel found
tshabong - nearest hotel: No hotel found
port saint john's - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
theunissen - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
sandnessjoen - nearest hotel: No hotel found
zeya - nearest hotel: No ho

,City,Country,Lat,Lng,Humidity,Hotel Name
2,lazaro cardenas,MX,17.9583,-102.2000,48,No hotel found
8,port alfred,ZA,-33.5906,26.8910,58,No hotel found
9,port elizabeth,ZA,-33.9180,25.5701,38,No hotel found
11,solleftea,SE,63.1667,17.2667,33,No hotel found
17,greeley,US,40.4233,-104.7091,59,No hotel found
...,...,...,...,...,...,...
567,ciudad lazaro cardenas,MX,17.9583,-102.2000,48,No hotel found
572,allariz,ES,42.1904,-7.8018,53,No hotel found
576,gao,ML,16.6362,1.6370,6,No hotel found
577,bolondron,CU,22.7633,-81.4469,32,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot_1 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles = "OSM",
    color='City',
    size='Humidity',
    hove_cols = ['Hotel Name','Country'])

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)